# Airlnie AI Assistant

## Imports & Setup

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# Updated function since course video

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are incredibly powerful feature provided by the frontier LLMs

With Tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky....we're giving it the power to run code on our machine?

Well, kinda.

### Write Function

In [5]:
# Let's start by making a useful function

ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499"
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

### Dictionary Structure to Describe Tool

In [7]:
# There's a particular dictionary structure that's required to describe our function

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

### Get OpenAI to use Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the oppurtunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        # AI saying it wants to run the tool
        message = response.choices[0].message
        # The response from the calling the tool
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [13]:
# We have to write handle_tool_call function

def handle_tool_call(message):
    # which tool it wants to call, in case there are several available
    tool_call = message.tool_calls[0]
    # what arguments it wants to call, returns json string so need json.loads
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    # call our function
    price = get_ticket_price(city)
    # This is what is added to messages, content is json dumps to convert to string
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_ticket_price called for Paris
Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for Timbuktu
